# 🤖 ChatBot Académico: Historia del Ecuador (Mini Embeddings Español)
Este notebook crea un chatbot educativo usando un archivo pequeño de embeddings en español, compatible con Google Colab y Google Drive.

In [ ]:
# ✅ Paso 1: Instalar librerías necesarias
!pip install gradio openpyxl --quiet

In [ ]:
# ✅ Paso 2: Cargar embeddings desde Google Drive o subir manualmente + preguntas historia Ecuador
from google.colab import drive, files
import pandas as pd

opcion = input('¿Quieres cargar embeddings desde Drive (d) o subir manualmente (m)? [d/m]: ').strip().lower()

if opcion == 'd':
    drive.mount('/content/drive')
    embeddings_path = input('Ingresa la ruta completa del archivo embeddings .vec en tu Drive: ')
elif opcion == 'm':
    uploaded = files.upload()
    embeddings_path = next(iter(uploaded))
else:
    raise ValueError('Opción inválida. Usa "d" para Drive o "m" para Manual.')

¿Quieres cargar embeddings desde Drive (d) o subir manualmente (m)? [d/m]: m


Saving Embeddings_Historia_Ecuador_placeholder_1024d.vec to Embeddings_Historia_Ecuador_placeholder_1024d.vec


In [ ]:
# ✅ Paso 3: Cargar los embeddings en memoria
import numpy as np

embeddings = {}
with open(embeddings_path, encoding='utf-8') as f:
    next(f)  # saltar primera línea (header)
    for line in f:
        valores = line.rstrip().split(' ')
        palabra = valores[0]
        vector = np.asarray(valores[1:], dtype='float32')
        embeddings[palabra] = vector
print(f"✅ Embeddings cargados: {len(embeddings)} palabras.")

✅ Embeddings cargados: 3044 palabras.


In [ ]:
# ✅ Paso 4: Subir archivo Excel con preguntas
uploaded = files.upload()
df = pd.read_excel(next(iter(uploaded)))
df = df.dropna()
df.head()

Saving Preguntas_Historia_Ecuador_Varias.xlsx to Preguntas_Historia_Ecuador_Varias.xlsx


,Pregunta,Respuesta
0,¿Qué se celebra el 10 de agosto en Ecuador?,El Primer Grito de Independencia en Quito.
1,¿En qué batalla se consolidó la independencia ...,En la Batalla de Pichincha.
2,¿Quién lideró la independencia de Guayaquil?,José Joaquín de Olmedo.
3,¿Qué año fue la independencia de Guayaquil?,En 1820.
4,¿Qué ciudad fue clave para la independencia de...,Quito.


In [ ]:
# ✅ Paso 5: Función para convertir frase a vector promedio
def frase_a_vector(frase):
    tokens = [pal.lower() for pal in frase.split() if pal.lower() in embeddings]
    if not tokens:
        return np.zeros(300)
    return np.mean([embeddings[pal] for pal in tokens], axis=0)

In [ ]:
# ✅ Paso 6: Vectorizar todas las preguntas
df['vector'] = df['Pregunta'].apply(frase_a_vector)

In [ ]:
# ✅ Paso 7: Función para responder la mejor pregunta
from sklearn.metrics.pairwise import cosine_similarity

def responder_usuario(pregunta):
    vec_preg = frase_a_vector(pregunta)
    similitudes = cosine_similarity([vec_preg], df['vector'].to_list())[0]
    idx = np.argmax(similitudes)
    if similitudes[idx] < 0.4:
        return "Lo siento, no tengo una respuesta para esa pregunta."
    return df.iloc[idx]['Respuesta']

In [ ]:
# ✅ Paso 8: Crear cliente Gradio
import gradio as gr
gr.Interface(
    fn=responder_usuario,
    inputs=gr.Textbox(label='Pregunta sobre Historia del Ecuador'),
    outputs=gr.Textbox(label='Respuesta del ChatBot'),
    title='🤖 ChatBot de Historia (Mini GloVe)',
    description='Chatbot que utiliza embeddings livianos en español para responder preguntas educativas.'
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd66e7dfb72fedf13d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
